# Code for Pulling Lyrics from Genius

In [20]:
from lyricsgenius import Genius
import os
from dotenv import load_dotenv
load_dotenv()
import pandas as pd
import sqlite3
conn = sqlite3.connect('lyrics.db')

In [14]:
#Lyrcis
genius = Genius(os.getenv('GENIUS_KEY'))

#need to write function to generate for each and store in a datastructure to not duplicate searches.

In [15]:
final_df = pd.read_csv("../datasets/Final database.csv")
full_df = pd.read_csv("../datasets/Database to calculate popularity.csv")


In [18]:
holder_df = full_df[full_df['country']=='USA']
holder_df.head()

,Unnamed: 0,country,date,position,uri,track,title,artist
280200,280200,USA,05/11/2020,1.0,https://open.spotify.com/track/35mvY5S1H3J2QZy...,"positions , Ariana Grande",positions,Ariana Grande
280201,280201,USA,05/11/2020,2.0,https://open.spotify.com/track/7hxHWCCAIIxFLCz...,"Lemonade , Internet Money",Lemonade,Internet Money
280202,280202,USA,05/11/2020,3.0,https://open.spotify.com/track/6Im9k8u9iIzKMrm...,"34+35 , Ariana Grande",34+35,Ariana Grande
280203,280203,USA,05/11/2020,4.0,https://open.spotify.com/track/47EiUVwUp4C9fGc...,"Dakiti , Bad Bunny, Jhay Cortez",Dakiti,"Bad Bunny, Jhay Cortez"
280204,280204,USA,05/11/2020,5.0,https://open.spotify.com/track/3tjFYV6RSFtuktY...,"Mood , 24kGoldn",Mood,24kGoldn


In [23]:
def storeLyrics(songName,artistName): #sep and local
    
    artist = genius.search_artist(artistName, max_songs=1)
    song = genius.search_song(songName, artist.name)
    lyrics = song.lyrics
    query = 'INSERT INTO lyrics VALUES ('+songName+','+artistName+','+lyrics+')'
    conn.execute(query)
    #return lyrics

In [24]:
storeLyrics("Dakiti","Bad Bunny, Jhay Cortez")
conn.close()

Searching for songs by Bad Bunny, Jhay Cortez...

Changing artist name to 'Bad Bunny & Jhay Cortez'
Song 1: "DÁKITI"

Reached user-specified song limit (1).
Done. Found 1 songs.
Searching for "Dakiti" by Bad Bunny & Jhay Cortez...
Done.


OperationalError: unrecognized token: "{"